In [1]:
import os
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm


%matplotlib inline

In [2]:
data_path = "data/"
check_path = "checkpoints/"

# os.makedirs(check_path)

os.chdir("../")
print(os.getcwd())

/Users/flynnzhang/CMU/Spring24/02620-ML4Scientists/scRNA-seq_ML


# Regression

#### List of cell types that are worth exploring:
- Immune cells
- Fibroblasts
- Endothelial cells
- Epithelial cells

In [4]:
adata = sc.read_h5ad(check_path + 'clustered.h5ad')

adata

AnnData object with n_obs × n_vars = 23873 × 2723
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'batch_colors', 'hvg', 'kmeans_clusters_colors', 'log1p', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [12]:
# load all tfs
tfs = pd.read_csv(data_path + "TFs/tfs.txt", sep="\t")

tfs

,Gene_symbol,Gene_Name,Synonyms,Human_entrez,Mouse_entrez,Rat_entrez
0,A430033K04Rik,-,-,NaN,243308.0,304336.0
1,AASS,"Alpha-aminoadipic semialdehyde synthase, mitoc...",LORSDH LKRSDH,10157.0,NaN,NaN
2,AATF,Protein AATF,DED CHE-1 CHE1 BFR2,26574.0,56321.0,114512.0
3,ABL1,Tyrosine-protein kinase ABL1,JTK7 c-ABL p150,25.0,11350.0,100909750.0
4,ABRA,Actin-binding Rho-activating protein,STARS,137735.0,223513.0,286965.0
...,...,...,...,...,...,...
3549,ZXDA,Zinc finger X-linked protein ZXDA,ZNF896,7789.0,668166.0,NaN
3550,ZXDB,Zinc finger X-linked protein ZXDB,ZNF905,158586.0,668166.0,NaN
3551,ZXDC,Zinc finger protein ZXDC,MGC11349 FLJ13861,79364.0,80292.0,362399.0
3552,ZYX,Zyxin,-,7791.0,NaN,NaN


In [23]:
# we need mouse tfs
TFs_mouse = tfs[tfs['Mouse_entrez'].notna()]['Gene_symbol'].values
TFs_mouse = [x for x in TFs_mouse if x in adata.var_names]

print('Total genes:', len(adata.var_names))
print('Total present TFs:', len(TFs_mouse))

Total genes: 2723
Total present TFs: 4


## Linear Regression

In [38]:
import statsmodels.api as sm

# 'TFs_mouse': list of TFs in mouse
def OLS(adata, tf_list):
    results = {}
    
    for tf in tf_list:
        Y = adata[:, tf].X
        X = adata.X[:, ~adata.var_names.isin([tf])] # remove the TF from the gene list
        X = sm.add_constant(X) # add constant term

        model = sm.OLS(Y, X)
        results[tf] = model.fit() # save the results for each model
        
    return results

OLS_res = OLS(adata, TFs_mouse)

In [39]:
# see the results
print('TFs:', OLS_res.keys())
# print all r-squared values
print('R-squared:', [OLS_res[tf].rsquared for tf in OLS_res.keys()])

TFs: dict_keys(['Csprs', 'Fgf2', 'Rhox5', 'Trib2'])
R-squared: [0.43436670324187987, 0.43202000073937996, 0.1510256947215608, 0.40364636670694787]
